#### **Optimización de Datos**: Conversión a Parquet.

En el proceso de manejo de datos, primero cargo los archivos credits.csv y movies_dataset.csv en DataFrames, los convierto a formato Parquet para optimizar el rendimiento y, luego, elimino los archivos CSV originales. Posteriormente, trabajo con los archivos Parquet en todas las fases del proyecto para asegurar un manejo más eficiente de los datos.

#### **Importo las Librerías Necesarias**

In [1]:
import pandas as pd             # Limpieza y manipulación de datos tabulares.
import json                     # Manejo de columnas anidadas en formato JSON.
import pyarrow.parquet as pq    # Almacenar y leer datos eficientemente.
import numpy as np              # Cálculos numéricos y normalización de datos.
import os                       # Gestionar archivos y directorios.
import ast                      # Convertir texto en estructuras de datos útiles.
import matplotlib.pyplot as plt # Crear gráficos para análisis exploratorio.

#### **Cargo los archivos CSV**

In [ ]:
# Leer el archivo credits.csv
credits_csv_df = pd.read_csv('Datasets/credits.csv')

# Leer el archivo movies_dataset.csv
movies_csv_df = pd.read_csv('Datasets/movies_dataset.csv')

# Mostrar las primeras filas de cada DataFrame
print("* Datos de credits:")
print(credits_csv_df.head())

print("\n* Datos de movies_dataset:")
print(movies_csv_df.head())

#### **Convertir `movies_dataset.csv` y `credits.csv` a Formato Parquet**

**Para movies_dataset** 

In [10]:
# Convertir la columna 'popularity' a tipo float, manejando errores
movies_csv_df['popularity'] = pd.to_numeric(movies_csv_df['popularity'], errors='coerce')

# Guardar los DataFrames en formato Parquet con nombres únicos
credits_csv_df.to_parquet('Datasets/credits_formato.parquet', index=False)
movies_csv_df.to_parquet('Datasets/movies_dataset_formato.parquet', index=False)

print("* Archivos guardados en formato Parquet con nombres diferenciados.")

* Archivos guardados en formato Parquet con nombres diferenciados.




Durante la conversión a formato Parquet, encontré un error relacionado con la columna **popularity**. Al investigar, descubrí que algunos valores eran de tipo texto en lugar de numéricos, lo que impedía la conversión adecuada. Para corregir esto, identifiqué y eliminé los valores no numéricos, y luego convertí la columna a tipo float para asegurar la compatibilidad. Tras esta limpieza, pude convertir el dataset a Parquet sin problemas, optimizándolo y dejándolo listo para el análisis.

#### **Verificar que no hay valores NaN en la columna popularity de csv**

In [11]:
# Revisar cuántos valores son NaN en la columna 'popularity' después de la conversión
print("Cantidad de valores NaN en 'popularity':", movies_csv_df['popularity'].isna().sum())

# Mostrar algunos ejemplos de valores NaN si existen
valores_nan = movies_csv_df[movies_csv_df['popularity'].isna()]
print("\nEjemplos de filas con valores NaN en 'popularity':")
print(valores_nan.head())


Cantidad de valores NaN en 'popularity': 6

Ejemplos de filas con valores NaN en 'popularity':
                                                   adult  \
19729                                              False   
19730                                 - Written by Ørnås   
29502                                              False   
29503   Rune Balot goes to a casino connected to the ...   
35586                                              False   

                                   belongs_to_collection  \
19729                                                NaN   
19730                                           0.065736   
29502  {'id': 122661, 'name': 'Mardock Scramble Colle...   
29503                                           1.931659   
35586                                                NaN   

                                 budget  \
19729                                 0   
19730  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29502                                 0   
29503  /zV8

In [12]:
# Eliminar filas donde 'popularity' sea NaN
movies_csv_df = movies_csv_df.dropna(subset=['popularity'])

# Verificar nuevamente cuántos valores NaN quedan en 'popularity'
print("Cantidad de valores NaN en 'popularity' después de la eliminación:", movies_csv_df['popularity'].isna().sum())

Cantidad de valores NaN en 'popularity' después de la eliminación: 0


In [13]:
# Guardar el DataFrame actualizado en formato Parquet
movies_csv_df.to_parquet('Datasets/movies_dataset_formato.parquet', index=False)

print("* Archivo guardado en formato Parquet después de eliminar NaNs en 'popularity'.")

* Archivo guardado en formato Parquet después de eliminar NaNs en 'popularity'.


**Cargar y Analizar Tamaño de Archivos Parquet**

Defino las rutas y leo los archivos Parquet para evaluar su tamaño en MB.

In [14]:
# Rutas de los archivos originales
movies_original_path = 'Datasets/movies_dataset_formato.parquet'
credits_original_path = 'Datasets/credits_formato.parquet'

# Cargar los archivos Parquet originales
original_movies_df = pd.read_parquet(movies_original_path)
original_credits_df = pd.read_parquet(credits_original_path)

# Función para obtener el tamaño del archivo
def get_file_size(file_path):
    return os.path.getsize(file_path)

# Imprimir el tamaño de cada archivo
print(f"Tamaño de 'original_movies_df': {get_file_size(movies_original_path)} bytes")
print(f"Tamaño de 'original_credits_df': {get_file_size(credits_original_path)} bytes")


Tamaño de 'original_movies_df': 17207877 bytes
Tamaño de 'original_credits_df': 68441610 bytes


#### **Visualización Inicial de Datos**

In [15]:
# Mostrar las primeras 5 filas del DataFrame de películas
print("* Primeras 5 filas de 'movies_dataset':")
original_movies_df.head()

* Primeras 5 filas de 'movies_dataset':


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,None,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",None,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",None,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,None,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",None,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",None,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
# Mostrar las primeras 5 filas del DataFrame de créditos
print("* Primeras 5 filas de 'credits_dataset':")
original_credits_df.head()

* Primeras 5 filas de 'credits_dataset':


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


**Observaciones:**

Al revisar ambos datasets, noto que movies contiene información esencial sobre las películas, como el título, la fecha de lanzamiento, el presupuesto y los ingresos, permitiendo el análisis del desempeño financiero. Por otro lado, credits detalla el elenco y el equipo, útil para identificar actores y directores y analizar su influencia en el éxito de las películas. Ambos datasets se pueden unir mediante la columna id, proporcionando una visión integral del contexto de cada película. Seguiré con las transformaciones necesarias.

#### **Elimino los archivos CSV**

He decidido eliminar los archivos CSV credits.csv y movies_dataset.csv ahora que he cargado sus datos en los DataFrames correspondientes. Para hacerlo de manera segura, utilizo el módulo os de Python, que me permite interactuar con el sistema de archivos. Aquí está el código que utilizo:

In [17]:
# Definir las rutas de los archivos CSV
credits_file_path = 'Datasets/credits.csv'
movies_file_path = 'Datasets/movies_dataset.csv'

# Verificar si el archivo existe antes de eliminarlo y eliminarlo
if os.path.exists(credits_file_path):
    os.remove(credits_file_path)
    print(f"* {credits_file_path} ha sido eliminado.")
else:
    print(f"* {credits_file_path} no existe.")

if os.path.exists(movies_file_path):
    os.remove(movies_file_path)
    print(f"* {movies_file_path} ha sido eliminado.")
else:
    print(f"* {movies_file_path} no existe.")

* Datasets/credits.csv ha sido eliminado.
* Datasets/movies_dataset.csv ha sido eliminado.


#### **Cargo los archivos Parquet originales**

In [18]:
# Rutas de los archivos originales
movies_original_path = 'Datasets/movies_dataset_formato.parquet'
credits_original_path = 'Datasets/credits_formato.parquet'

# Cargar los archivos Parquet originales
movies_df_original = pd.read_parquet(movies_original_path)
credits_df_original = pd.read_parquet(credits_original_path)

# Mostrar las primeras filas de cada DataFrame para verificar que se cargaron correctamente
print("* Datos de movies_df_original:")
print(movies_df_original.head())

print("\n* Datos de credits_df_original:")
print(credits_df_original.head())

* Datos de movies_df_original:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                               None  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                               None  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                  